In [ ]:
# !pip install mesa

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import tqdm

%load_ext nb_black

In [ ]:
# https://mesa.readthedocs.io/en/stable/

## To-Do:

* Optimierung: kriegen wirs besser hin. Benchmarks in Netlogo? 
* Faithfulness: Inwieweit implementieren wir eigentlich dasselbe wie in der Referenz/im Originalpaper? Antwort: garnicht, Strategien sind GEWICHTE!
* Agent.py https://agentpy.readthedocs.io/en/latest/
* Syntax für Gruppen? Gruppensyntax als Klasse? (Visualisierung als Methode)
* Visualierung: phylogenetic tree, bubblechart, treemap
* **Gruppen umsetzen** 
* UMAP embedding von Strategien, Visualisierung Bokeh, Faerun?
* Evaluierungsfunktionen, visualisierungen
* ABMs in R?
* Gitlab der Uni verwenden? 
* Package-integration mit Mesa recherchieren
* Spione
* bayesian updating statt vektor-vorraussage
* https://www.python-graph-gallery.com/circular-packing-several-levels-of-hierarchy


In [ ]:
# Gruppenstruktur:

# [[[a, b], [c, d]], e, f]

In [ ]:
# implement an artificial history.
# find out whether there is a running tally for the strategies, or whether they are just

In [ ]:
from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector


class MyAgent(Agent):
    def __init__(
        self,
        name,
        model,
        memory=10,
        n_strategies=10,
        threshold=50,
        history=[0, 0],
    ):

        super().__init__(name, model)

        self.name = name
        self.memory = memory
        self.n_strategies = n_strategies
        self.threshold = threshold
        self.history = history
        self.strategies = [self.random_strategy() for x in range(0, n_strategies)]
        self.best_strategy = self.strategies[0]
        self.best_strategy_no = 0
        self.update_strategies()

        current_prediction = self.predict_attendance(
            self.best_strategy, self.model.history[-self.memory * 2 :]
        )
        #         print(current_prediction) # predictions can be negative...

        if current_prediction <= threshold:
            self.should_i_attend = True
        else:
            self.should_i_attend = False

    #         print(self.random_strategy(self.memory))

    def random_strategy(self):
        return list(1 - np.random.rand(self.memory) * 2)

    def predict_attendance(self, strategy, subhistory):
        return 100 * strategy[0] + np.sum(
            [weight * week for weight, week in zip(strategy, subhistory)]
        )

    def update_strategies(self):
        best_score = self.memory * 100 + 1
        sub_history = self.model.history[-self.memory * 2 :]
        for strategy_no, this_strategy in enumerate(self.strategies):
            #             print("strategy_n ", strategy_no, "\n")
            score = 0

            for week in range(0, self.memory):  # bit unpythonic?
                prediction = self.predict_attendance(
                    this_strategy, sub_history[week : (week + self.memory)]
                )
                score = score + np.abs(sub_history[week - 1] - prediction)
            #                 print("this_strategies_score", score)

            if score <= best_score:
                best_score = score
                self.best_strategy = this_strategy
                self.best_strategy_no = strategy_no

    #             print("best_strategy ", self.best_strategy_no)

    #             return self

    def step(self):

        #         print("agent name ", self.name)

        self.update_strategies()
        #         if self.name == 0:
        #             print(self.best_strategy_no)
        #             print(self.best_strategy)
        #         print(self.model.history[-self.memory :])
        #         print(self.best_strategy_no)

        current_prediction = self.predict_attendance(
            self.best_strategy, self.model.history[-self.memory * 2 :]
        )
        if current_prediction <= self.threshold:
            self.should_i_attend = True
        else:
            self.should_i_attend = False


class MyModel(Model):
    def __init__(self, n_agents, memory, n_strategies, threshold):
        super().__init__()
        self.schedule = RandomActivation(self)

        self.history = list(np.random.rand(memory * 2) * 100)
        self.attendance = self.history[-1]

        for i in range(n_agents):
            a = MyAgent(i, self, memory, n_strategies, threshold, self.history)
            self.schedule.add(a)
        print("set up")

        agent_reporters = {
            "should_i_attend": lambda a: getattr(a, "should_i_attend", None),
            "best_strategy": lambda a: getattr(a, "best_strategy", None),
            "best_strategy_no": lambda a: getattr(a, "best_strategy_no", None),
        }

        model_reporters = {
            "attendance": lambda a: getattr(a, "attendance", None),
        }

        self.datacollector = DataCollector(
            model_reporters=model_reporters, agent_reporters=agent_reporters
        )

    def step(self):
        self.attendance = np.sum(
            [int(a.should_i_attend) for a in model.schedule.agents]
        )

        self.history.append(self.attendance)
        self.datacollector.collect(self)
        self.schedule.step()


model = MyModel(100, memory=5, n_strategies=10, threshold=60)


# Do the run:
for t in tqdm.tqdm_notebook(range(400)):
    model.step()

In [ ]:
model_df = model.datacollector.get_model_vars_dataframe()
agent_df = model.datacollector.get_agent_vars_dataframe()

In [ ]:
fig, ax = plt.subplots(figsize=(16, 6))
plt.style.use("opinions.mplstyle")
plt.plot(model_df["attendance"], linewidth=1.6, c="#ab0b00", alpha=1)
plt.xlabel("Time")
plt.ylabel("Attendence")
plt.title("Attendence in El Farol")
plt.ylim(0,100)
# fig.suptitle("Attendence in El Farol")


In [ ]:
agent_df

In [ ]:
agent_df["agent"] = agent_df.index.get_level_values("AgentID")
agent_df.droplevel(1)

In [ ]:
# agent_df = agent_df.dropna(subset=["selected_strategy"])

In [ ]:
# agent_df

In [ ]:
agent_df["strat_key"] = [
    x + "_" + y
    for x, y in zip(
        agent_df["agent"].astype(str),
        agent_df["best_strategy_no"].astype(int).astype(str),
    )
]



In [ ]:
agent_df

In [ ]:
# strat_played = []
# for ix, row in agent_df.iterrows():
#     strat_played.append(row["my_strategies"][int(row["best_strategy_no"]), :])
agent_df["strat_played"] = agent_df["best_strategy"]

In [ ]:
played_counts = (
    agent_df[["strat_key", "strat_played"]]
    .groupby(["strat_key"])["strat_key"]
    .transform("count")
)
agent_df["played_counts"] = played_counts

In [ ]:
used_strategy_df = agent_df.drop_duplicates(subset=["strat_key"])

In [ ]:
used_strategy_array = np.vstack([x for x in used_strategy_df["strat_played"]])

In [ ]:
used_strategy_df

In [ ]:
used_strategy_df["best_strategy_no"].value_counts()

In [ ]:
import umap
reducer = umap.UMAP(metric = 'cosine',n_neighbors=40, verbose= True)

embedding = reducer.fit_transform(used_strategy_array)


In [ ]:
plt.style.use("opinions.mplstyle")
fig, ax = plt.subplots(figsize=(9, 9))

plt.scatter(
    embedding[:, 0],
    embedding[:, 1],
    c=used_strategy_df["played_counts"],
    s=used_strategy_df["played_counts"],
    cmap="viridis",
    alpha=0.5,
)

In [ ]:
!rm -rf strategy_heatmaps
!mkdir strategy_heatmaps

In [ ]:
used_strategy_array.shape

In [ ]:
import cmocean

for ix, row in tqdm.tqdm_notebook(enumerate(used_strategy_array)):

    fig, ax = plt.subplots(figsize=(5, 0.5))

    sns.heatmap(row.reshape(1, -1), cmap=cmocean.cm.solar, cbar=False)
    ax.axes.get_xaxis().set_visible(False)
    ax.axes.get_yaxis().set_visible(False)
    plt.savefig(
        "strategy_heatmaps" + "/" + used_strategy_df["strat_key"].iloc[ix] + ".png"
    )
    plt.close()
#     print("strategy_heatmaps" + "/" + used_strategy_df["strat_key"].iloc[ix] + ".png")

In [ ]:
# used_strategy_df['strat_key']

In [ ]:
from bokeh.plotting import figure, output_file, show, ColumnDataSource
from bokeh.models import HoverTool
from bokeh.models import LinearColorMapper

color_mapper = LinearColorMapper(
    palette="Magma256",
    low=min(used_strategy_df["played_counts"]),
    high=max(used_strategy_df["played_counts"]),
)


output_file("toolbar.html")

source = ColumnDataSource(
    data=dict(
        x=embedding[:, 0],
        y=embedding[:, 1],
        imgs=[
            "strategy_heatmaps" + "/" + x + ".png"
            for x in used_strategy_df["strat_key"]
        ],
        s=(used_strategy_df["played_counts"] / 10) + 10,
        c=used_strategy_df["played_counts"],
    )
)

hover = HoverTool(
    tooltips="""
<div>
        <img
            src="@imgs" alt="@imgs" 
            style="float: left; margin: 0px 15px 15px 0px;"
            border="2"
        ></img>
</div>
"""
)

p = figure(
    plot_width=900,
    plot_height=900,
    tools=[hover],
    title="Mouse over the dots",
)

p.circle(
    "x",
    "y",
    size="s",
    #     color="c",
    alpha=0.7,
    fill_color={"field": "c", "transform": color_mapper},
    source=source,
)

show(p)

In [ ]:
def moving_average(x, w):
    return np.convolve(x, np.ones(w), "valid") / w


fig, ax = plt.subplots(figsize=(12, 6))
plt.style.use("opinions.mplstyle")
for agent_no in range(0, 10):
    agent_attendance = agent_df.iloc[
        agent_df.index.get_level_values("AgentID") == agent_no
    ]["should_i_attend"].astype(
        int
    )  # [0:200]

    plt.plot(moving_average(agent_attendance, 20), alpha=0.3, c="grey")

In [ ]:
list(range(0, 10))[-1]